# AI Travel Planner: Multi-Agent System with Route Optimization & Travel Insights

This notebook implements a multi-agent system for comprehensive travel planning:

### Core Concept: 
Coordinates specialized AI agents to handle different aspects of travel planning through a central manager agent

**Manager Agent**: Orchestrates the entire planning process by delegating tasks to specialized agents and synthesizing their outputs

**Specialized Agents**:

* Destination Research Agent: Gathers location information and highlights
* Travel Ticket Search Agent: Finds optimal transportation between cities
* Weather Forecast Agent: Analyzes ideal travel seasons
* Local Route Travel Agent: Optimizes travel between attractions
* Budget Planner Agent: Estimates comprehensive trip costs
* Final Itinerary Generator: Compiles all information into a structured plan


### Tools Integration: 
Leverages Google/DuckDuckGo search, web scraping, and Google Maps API for real-time data

### Key Features: 
Multi-location optimization, real-time pricing, budget awareness, and flexible transportation options


In [1]:
%pip install -qU smolagents requests pandas  googlemaps langchain-community dotenv


Note: you may need to restart the kernel to use updated packages.


## OpenAI Model

In [2]:
import os
from smolagents import OpenAIServerModel
from dotenv import load_dotenv

load_dotenv()
model = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

/Users/dangphuonganh/opt/anaconda3/envs/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Destination research agent

In [3]:
from smolagents import GoogleSearchTool
search_tool =GoogleSearchTool()

### GoogleSearchTool

More up-to-dated search results than DuckDuck

In [4]:
import os
from smolagents import GoogleSearchTool, ToolCallingAgent
from dotenv import load_dotenv
load_dotenv()

os.environ["SERPAPI_API_KEY"] = os.getenv('SERPAPI_API_KEY') 

#https://serpapi.com/manage-api-key
destination_research_agent = ToolCallingAgent(
    tools=[GoogleSearchTool()],
    model=model,
    name="destination_research_agent",
    description="Finds top destinations, best times to visit, and city highlights. Use most recent/updated data",
    max_steps=2
)



### DuckDuckGoSearchTool

Cheaper than GoogleSearch

In [5]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, VisitWebpageTool

destination_research_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="destination_research_agent",
    description="Finds top destinations, best times to visit, and city highlights.",
    max_steps=2
)

# Route optimization agent

In [14]:

from typing import Optional
from smolagents import CodeAgent, tool

@tool
def get_travel_duration(start_location: str, destination_location: str, mode: Optional[str] = None, departure_time: Optional[int] = None) -> str:
    """Gets the travel time in car between two places.
    
    Args:
        start_location: the place from which you start your ride
        destination_location: the place of arrival
        departure_time: the departure time, provide only a datetime.datetime if you want to specify this
        mode: The transportation mode, in 'driving', 'walking', 'bicycling', or 'transit'. Defaults to 'driving'.
              if mode is 'transit', the api automatically selects the best combination of transit modes (bus, subway, train, tram, etc.) based on what’s available and optimal for your route.
    """
    import googlemaps
    import os
    from datetime import datetime

    gmaps = googlemaps.Client(os.getenv("GMAPS_API_KEY"))

    if departure_time is None:
        departure_time = datetime.now()

    directions_result = gmaps.directions(
        start_location,
        destination_location,
        mode="transit",
        departure_time=departure_time
    )
    return directions_result[0]["legs"][0]["duration"]["text"]

# Initialize the CodeAgent
local_route_travel_agent = CodeAgent(
    name="local_route_travel_agent",
    tools=[get_travel_duration], 
    model=model,  
    additional_authorized_imports=["datetime"],
    description=(
    "Use this agent when the user needs to plan travel between multiple places within the same city or between nearby cities. "
    "Select the best travel mode such as walking, driving, or public transport. "
    "Takes into account travel time, user preferences, and distance."
)

      #How many times can the agent write/run code before giving up?
)


In [17]:
travel_times = local_route_travel_agent.run("Travel times and modes between Hoan Kiem Lake, Old Quarter, Ho Chi Minh Mausoleum, Temple of Literature, and One Pillar Pagoda.")                                               
print(travel_times)                                                                                              

╭────────────────────────────────────── New run - local_route_travel_agent ───────────────────────────────────────╮
│                                                                                                                 │
│ Travel times and modes between Hoan Kiem Lake, Old Quarter, Ho Chi Minh Mausoleum, Temple of Literature, and    │
│ One Pillar Pagoda.                                                                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  locations = ["Hoan Kiem Lake", "Old Quarter", "Ho Chi Minh Mausoleum", "Temple of Literature", "One Pillar       
  Pagoda"]                                                                                                         
  travel_times = {}                                                                                                
                                                                                                                   
  for start in locations:                                                                                          
      for destination in locations:                                                                                
          if start != destination:                                                                                 
              duration = get_travel_duration(start_location=start, destination_location=destination,               
  mode='driving')                                                                                                  
              travel_times[(start, destination)] = duration                                                        
                                                                                                                   
  print(travel_times)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{('Hoan Kiem Lake', 'Old Quarter'): '17 mins', ('Hoan Kiem Lake', 'Ho Chi Minh Mausoleum'): '22 mins', ('Hoan Kiem 
Lake', 'Temple of Literature'): '29 mins', ('Hoan Kiem Lake', 'One Pillar Pagoda'): '20 mins', ('Old Quarter', 
'Hoan Kiem Lake'): '18 mins', ('Old Quarter', 'Ho Chi Minh Mausoleum'): '20 mins', ('Old Quarter', 'Temple of 
Literature'): '33 mins', ('Old Quarter', 'One Pillar Pagoda'): '17 mins', ('Ho Chi Minh Mausoleum', 'Hoan Kiem 
Lake'): '25 mins', ('Ho Chi Minh Mausoleum', 'Old Quarter'): '23 mins', ('Ho Chi Minh Mausoleum', 'Temple of 
Literature'): '17 mins', ('Ho Chi Minh Mausoleum', 'One Pillar Pagoda'): '4 mins', ('Temple of Literature', 'Hoan 
Kiem Lake'): '30 mins', ('Temple of Literature', 'Old Quarter'): '25 mins', ('Temple of Literature', 'Ho Chi Minh 
Mausoleum'): '17 mins', ('Temple of Literature', 'One Pillar Pagoda'): '16 mins', ('One Pillar Pagoda', 'Hoan Kiem 
Lake'): '22 mins', ('One Pillar Pagoda', 'Old Quarter'): '21 mins', ('One Pillar Pagoda', 'Ho Chi Minh Mausoleum'):
'4 mins', ('One Pillar Pagoda', 'Temple of Literature'): '16 mins'}

Out: None

[Step 1: Duration 12.10 seconds| Input tokens: 2,173 | Output tokens: 223]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Travel times between locations in Hanoi:\n"                                                        
      "Hoan Kiem Lake to Old Quarter: 17 mins\n"                                                                   
      "Hoan Kiem Lake to Ho Chi Minh Mausoleum: 22 mins\n"                                                         
      "Hoan Kiem Lake to Temple of Literature: 29 mins\n"                                                          
      "Hoan Kiem Lake to One Pillar Pagoda: 20 mins\n"                                                             
      "Old Quarter to Hoan Kiem Lake: 18 mins\n"                                                                   
      "Old Quarter to Ho Chi Minh Mausoleum: 20 mins\n"                                                            
      "Old Quarter to Temple of Literature: 33 mins\n"                                                             
      "Old Quarter to One Pillar Pagoda: 17 mins\n"                                                                
      "Ho Chi Minh Mausoleum to Hoan Kiem Lake: 25 mins\n"                                                         
      "Ho Chi Minh Mausoleum to Old Quarter: 23 mins\n"                                                            
      "Ho Chi Minh Mausoleum to Temple of Literature: 17 mins\n"                                                   
      "Ho Chi Minh Mausoleum to One Pillar Pagoda: 4 mins\n"                                                       
      "Temple of Literature to Hoan Kiem Lake: 30 mins\n"                                                          
      "Temple of Literature to Old Quarter: 25 mins\n"                                                             
      "Temple of Literature to Ho Chi Minh Mausoleum: 17 mins\n"                                                   
      "Temple of Literature to One Pillar Pagoda: 16 mins\n"                                                       
      "One Pillar Pagoda to Hoan Kiem Lake: 22 mins\n"                                                             
      "One Pillar Pagoda to Old Quarter: 21 mins\n"                                                                
      "One Pillar Pagoda to Ho Chi Minh Mausoleum: 4 mins\n"                                                       
      "One Pillar Pagoda to Temple of Literature: 16 mins"                                                         
  )                                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Travel times between locations in Hanoi:
Hoan Kiem Lake to Old Quarter: 17 mins
Hoan Kiem Lake to Ho Chi Minh Mausoleum: 22 mins
Hoan Kiem Lake to Temple of Literature: 29 mins
Hoan Kiem Lake to One Pillar Pagoda: 20 mins
Old Quarter to Hoan Kiem Lake: 18 mins
Old Quarter to Ho Chi Minh Mausoleum: 20 mins
Old Quarter to Temple of Literature: 33 mins
Old Quarter to One Pillar Pagoda: 17 mins
Ho Chi Minh Mausoleum to Hoan Kiem Lake: 25 mins
Ho Chi Minh Mausoleum to Old Quarter: 23 mins
Ho Chi Minh Mausoleum to Temple of Literature: 17 mins
Ho Chi Minh Mausoleum to One Pillar Pagoda: 4 mins
Temple of Literature to Hoan Kiem Lake: 30 mins
Temple of Literature to Old Quarter: 25 mins
Temple of Literature to Ho Chi Minh Mausoleum: 17 mins
Temple of Literature to One Pillar Pagoda: 16 mins
One Pillar Pagoda to Hoan Kiem Lake: 22 mins
One Pillar Pagoda to Old Quarter: 21 mins
One Pillar Pagoda to Ho Chi Minh Mausoleum: 4 mins
One Pillar Pagoda to Temple of Literature: 16 mins

[Step 3: Duration 6.46 seconds| Input tokens: 8,489 | Output tokens: 730]

Travel times between locations in Hanoi:
Hoan Kiem Lake to Old Quarter: 17 mins
Hoan Kiem Lake to Ho Chi Minh Mausoleum: 22 mins
Hoan Kiem Lake to Temple of Literature: 29 mins
Hoan Kiem Lake to One Pillar Pagoda: 20 mins
Old Quarter to Hoan Kiem Lake: 18 mins
Old Quarter to Ho Chi Minh Mausoleum: 20 mins
Old Quarter to Temple of Literature: 33 mins
Old Quarter to One Pillar Pagoda: 17 mins
Ho Chi Minh Mausoleum to Hoan Kiem Lake: 25 mins
Ho Chi Minh Mausoleum to Old Quarter: 23 mins
Ho Chi Minh Mausoleum to Temple of Literature: 17 mins
Ho Chi Minh Mausoleum to One Pillar Pagoda: 4 mins
Temple of Literature to Hoan Kiem Lake: 30 mins
Temple of Literature to Old Quarter: 25 mins
Temple of Literature to Ho Chi Minh Mausoleum: 17 mins
Temple of Literature to One Pillar Pagoda: 16 mins
One Pillar Pagoda to Hoan Kiem Lake: 22 mins
One Pillar Pagoda to Old Quarter: 21 mins
One Pillar Pagoda to Ho Chi Minh Mausoleum: 4 mins
One Pillar Pagoda to Temple of Literature: 16 mins


# Travel Ticket Search Agent

In [7]:
flight_search_agent = ToolCallingAgent(
    tools=[GoogleSearchTool(), VisitWebpageTool()],
    model=model,
    name="travel_ticket_search_agent",
    # additional_authorized_imports=["time"],
    description=(
      "Finds best flights/ trains (other transportation) between cities or countries based on real-time ticketprices/ historical price trends, and best booking periods."
      "If a currency is not specified, use local currency (of the destination) or USD as default."
      "Always plan the trip in the future. Do not return expired or old tickets."
    ),
    max_steps=2 #How many times can the agent write/run code before giving up?
)


# Weather analysis agent

In [8]:
weather_forecast_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    name="weather_forecast_agent",
    description="Use this if no specific travel time is specified to finds best travel seasons by analyzing weather trends."
)


# Budget Planner Agent

In [9]:
from smolagents import CodeAgent
budget_planner_agent = CodeAgent(
    model=model,
    tools=[],
    name="budget_planner_agent",
    description=(
        "Estimates total travel cost for a trip. Takes into account flight/trains ticket prices (consider what is more suitable for budget), "
        "accommodation costs per night, local transportation, and optionally food or activities. "
        "Use this agent to provide an estimated total budget for the full trip, broken down by category."
        "If a currency is not specified, use USD as default."
    )
)

# Final Itinerary Generator


Compiles best travel dates, routes, and cost into a structured trip plan.

In [10]:
final_itinerary_agent = CodeAgent(
    model=model,
    name="final_itinerary_agent",
    tools=[],
    description=(
        "Compiles a complete travel plan. Takes input from research, flight/travel tickets, route, weather, and budget agents, "
        "and creates a structured itinerary including travel dates, locations, recommended routes, estimated costs, "
        "accommodation suggestions, daily activities, and helpful travel tips."
    )
)

# Travel Planner Manager Agent

In [18]:
import time

# find current time
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

travel_manager_agent = CodeAgent(
    model=model,
    managed_agents=[
        destination_research_agent,
        flight_search_agent,
        weather_forecast_agent,
       local_route_travel_agent,
        budget_planner_agent,
        final_itinerary_agent
    ],
    #tools=[DuckDuckGoSearchTool()],
    tools=[],
    additional_authorized_imports=["time"],
    name="travel_manager_agent",
    description=(
        "Coordinates a team of specialized agents to build a complete travel plan. "
        "Delegates tasks such as finding destinations, decide the best travel order (shortest & cheapest route), searching for flights, checking weather, planning routes, "
        "estimating costs, and compiling the final itinerary. Use this agent for end-to-end trip planning across multiple days or locations."
        "Always plan the trip in the future."
        "Provide the total budget in the specified currency if any else use USD as default."

    )  
)

In [19]:
user_request = """
Plan a 2-day trip in Hanoi with the following requirements:
I want to visit Hoan Kiem Lake, Old Quarter, Ho Chi Minh Mausoleum, Temple of Literature, and One Pillar Pagoda.
Find the best travel mode and estimated time between attractions.
Estimate the total budget. 
"""

response = travel_manager_agent.run(user_request)
print(response)

╭──────────────────────────────────────── New run - travel_manager_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ Plan a 2-day trip in Hanoi with the following requirements:                                                     │
│ I want to visit Hoan Kiem Lake, Old Quarter, Ho Chi Minh Mausoleum, Temple of Literature, and One Pillar        │
│ Pagoda.                                                                                                         │
│ Find the best travel mode and estimated time between attractions.                                               │
│ Estimate the total budget.                                                                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  attractions = [                                                                                                  
      "Hoan Kiem Lake",                                                                                            
      "Old Quarter",                                                                                               
      "Ho Chi Minh Mausoleum",                                                                                     
      "Temple of Literature",                                                                                      
      "One Pillar Pagoda"                                                                                          
  ]                                                                                                                
  travel_time = local_route_travel_agent(attractions=attractions)                                                  
  print(travel_time)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'travel_time = local_route_travel_agent(attractions=attractions)' due to: TypeError: 
MultiStepAgent.__call__() missing 1 required positional argument: 'task'

[Step 1: Duration 3.58 seconds| Input tokens: 2,390 | Output tokens: 173]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Itinerary": [                                                                                               
          "Day 1: Visit Hoan Kiem Lake, Old Quarter, and Ho Chi Minh Mausoleum.",                                  
          "Day 2: Visit Temple of Literature and One Pillar Pagoda."                                               
      ],                                                                                                           
      "Travel Modes": "Driving or walking (total loop time: 92 minutes)",                                          
      "Total Budget": "$170"                                                                                       
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Itinerary': ['Day 1: Visit Hoan Kiem Lake, Old Quarter, and Ho Chi Minh Mausoleum.', 'Day 2: 
Visit Temple of Literature and One Pillar Pagoda.'], 'Travel Modes': 'Driving or walking (total loop time: 92 
minutes)', 'Total Budget': '$170'}

[Step 5: Duration 5.55 seconds| Input tokens: 18,460 | Output tokens: 1,651]

{'Itinerary': ['Day 1: Visit Hoan Kiem Lake, Old Quarter, and Ho Chi Minh Mausoleum.', 'Day 2: Visit Temple of Literature and One Pillar Pagoda.'], 'Travel Modes': 'Driving or walking (total loop time: 92 minutes)', 'Total Budget': '$170'}


# Use Toolingcallingagent as the manager agents


CodeAgent often has issue with generating the correct code for calling the managed agents.

In [27]:
travel_manager_agent = ToolCallingAgent(
    managed_agents=[
        destination_research_agent,
        flight_search_agent,
        weather_forecast_agent,
       local_route_travel_agent,
        budget_planner_agent,
        final_itinerary_agent
    ],
    tools=[],
    model=model,
    name="travel_manager_agent",
    description=(
        "Coordinates a team of specialized agents to build a complete travel plan. "
        "Delegates tasks such as finding destinations, decide the best travel order (shortest & cheapest route), searching for flights, checking weather, planning routes, "
        "estimating costs, and compiling the final itinerary. Use this agent for end-to-end trip planning across multiple days or locations."
        "Always plan the trip in the future."
        "Provide the total budget in the specified currency if any else use USD as default."

    )  )


user_request = """
You must call the manager agents to plan the trip.
Plan a 1 day trip from Vietnam to Bangkok. 
List top tourist places
Find the best travel mode and estimated traveltime between attractions within the city.
Estimate the total budget. 
"""

response = travel_manager_agent.run(user_request)
print(response)


╭──────────────────────────────────────── New run - travel_manager_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ You must call the manager agents to plan the trip.                                                              │
│ Plan a 1 day trip from Vietnam to Bangkok.                                                                      │
│ List top tourist places                                                                                         │
│ Find the best travel mode and estimated traveltime between attractions within the city.                         │
│ Estimate the total budget.                                                                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'destination_research_agent' with arguments: {'task': 'Please find the top tourist attractions in │
│ Bangkok for a one-day trip from Vietnam.'}                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────── New run - destination_research_agent ──────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'destination_research_agent'.                                                      │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Please find the top tourist attractions in Bangkok for a one-day trip from Vietnam.                             │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'For a one-day trip from Vietnam to Bangkok, the top    │
│ tourist attractions include the Grand Palace, Wat Pho, Wat Arun, a canal tour, and visiting Khao San Road. The  │
│ total estimated travel time between all locations is about 54 minutes by driving. The average round-trip flight │
│ cost is approximately A$189, and the total estimated travel budget for the trip is 7452.50 THB. This budget     │
│ includes flight costs (4252.50 THB), entrance fees for attractions (2150 THB), local transportation (300 THB),  │
│ and food costs (750 THB).'}                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: For a one-day trip from Vietnam to Bangkok, the top tourist attractions include the Grand Palace, Wat
Pho, Wat Arun, a canal tour, and visiting Khao San Road. The total estimated travel time between all locations is 
about 54 minutes by driving. The average round-trip flight cost is approximately A$189, and the total estimated 
travel budget for the trip is 7452.50 THB. This budget includes flight costs (4252.50 THB), entrance fees for 
attractions (2150 THB), local transportation (300 THB), and food costs (750 THB).

[Step 5: Duration 3.05 seconds| Input tokens: 29,100 | Output tokens: 2,307]

For a one-day trip from Vietnam to Bangkok, the top tourist attractions include the Grand Palace, Wat Pho, Wat Arun, a canal tour, and visiting Khao San Road. The total estimated travel time between all locations is about 54 minutes by driving. The average round-trip flight cost is approximately A$189, and the total estimated travel budget for the trip is 7452.50 THB. This budget includes flight costs (4252.50 THB), entrance fees for attractions (2150 THB), local transportation (300 THB), and food costs (750 THB).


In [28]:
from pprint import pprint

pprint(response)

('For a one-day trip from Vietnam to Bangkok, the top tourist attractions '
 'include the Grand Palace, Wat Pho, Wat Arun, a canal tour, and visiting Khao '
 'San Road. The total estimated travel time between all locations is about 54 '
 'minutes by driving. The average round-trip flight cost is approximately '
 'A$189, and the total estimated travel budget for the trip is 7452.50 THB. '
 'This budget includes flight costs (4252.50 THB), entrance fees for '
 'attractions (2150 THB), local transportation (300 THB), and food costs (750 '
 'THB).')
